### Step1 - 导入相关包 & 初始化设置

In [ ]:
import os, sys

from langchain.chat_models import init_chat_model
# 常用的消息类型
from langchain.messages import HumanMessage, SystemMessage, ToolMessage, AIMessage, AIMessageChunk

# 默读取当前目录下的 .env 文件, 可以通过 dotenv_path 来修改
from dotenv import load_dotenv
root_dir = os.path.dirname(os.getcwd())
load_dotenv(dotenv_path=os.path.join(root_dir, ".env"))

# 记录日志
from loguru import logger
logger.remove()
logger.add(sys.stderr, level=os.getenv("LOG_LEVEL"))

1

### Step2 - 初始化聊天模型

In [2]:
# LangChain v1 创建聊天模型的方法
chat_model = init_chat_model(
    model_provider="ollama",
    # model 也可以写为 <model_provider>:<model_name> 的形式
    # 这样就可以不用指定 model_provider 这个参数了
    model = os.getenv("OLLAMA_LLM_MODEL"),
    base_url = os.getenv("OLLAMA_BASE_URL"),
    max_tokens = 1024,
)
chat_model

ChatOllama(model='qwen3:4b-instruct', base_url='http://localhost:11434')

### Step3 - 定义聊天模板

In [3]:
messages = [
    SystemMessage(content="你是一个专业的AI助手, 请使用简洁的语言来回答用户的问题."),
]
# 可以用 append 的方式来添加/保存对话信息
messages.append(HumanMessage(content="请简单介绍一下你自己."))
messages

[SystemMessage(content='你是一个专业的AI助手, 请使用简洁的语言来回答用户的问题.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='请简单介绍一下你自己.', additional_kwargs={}, response_metadata={})]

### Step4 - 执行对话

In [4]:
# 方法1: invoke 非流式输出
respone = chat_model.invoke(messages)

# 类型为: <class 'langchain_core.messages.ai.AIMessage'>
logger.debug(type(respone))

for msg in respone:
    print(msg)

2025-12-04 23:05:29.393 | DEBUG    | __main__:<module>:5 - <class 'langchain_core.messages.ai.AIMessage'>


('content', '你好！我是一个AI助手，专注于提供准确、简洁的信息和帮助解决问题。我可以回答问题、提供建议、协助写作，也可以陪你聊天。无论你有什么需求，我都会尽力为你服务！😊')
('additional_kwargs', {})
('response_metadata', {'model': 'qwen3:4b-instruct', 'created_at': '2025-12-04T15:05:29.3925582Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1101230200, 'load_duration': 181060300, 'prompt_eval_count': 34, 'prompt_eval_duration': 290296100, 'eval_count': 44, 'eval_duration': 611371700, 'logprobs': None, 'model_name': 'qwen3:4b-instruct', 'model_provider': 'ollama'})
('type', 'ai')
('name', None)
('id', 'lc_run--81105da1-a7b7-4cc9-b152-50c49166c6db-0')
('tool_calls', [])
('invalid_tool_calls', [])
('usage_metadata', {'input_tokens': 34, 'output_tokens': 44, 'total_tokens': 78})


In [5]:
# 方法2: steam 流式输出
display_debug = False

for chunk in chat_model.stream(messages):
    # 判断内容是否是 AIMessageChunk, 如果是则打印内容
    if isinstance(chunk, AIMessageChunk):
        print(chunk.content, end="", flush=True)
    # 展示其中一个 chunk 中的内容
    if display_debug is False:
        logger.debug(f"chunk type: {type(chunk)}")
        for i in chunk:
            print(i)
        display_debug = True



你好

2025-12-04 23:05:29.646 | DEBUG    | __main__:<module>:10 - chunk type: <class 'langchain_core.messages.ai.AIMessageChunk'>


('content', '你好')
('additional_kwargs', {})
('response_metadata', {})
('type', 'AIMessageChunk')
('name', None)
('id', 'lc_run--d5d5a59d-c74c-4958-aeb8-81b8d21f3b81')
('tool_calls', [])
('invalid_tool_calls', [])
('usage_metadata', None)
('tool_call_chunks', [])
('chunk_position', None)
！我是一个AI助手，专注于提供准确、简洁的信息和帮助解决各种问题。我可以回答你的疑问、提供建议、协助写作或进行逻辑推理。无论你有什么需求，我都会尽力以清晰、可靠的方式回应你。有什么我可以帮你的吗？